# Librerías

In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
from collections import Counter
import re
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pprint
from nltk import FreqDist
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import gensim
from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary
from sklearn.feature_extraction.text import HashingVectorizer, TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.preprocessing.text import Tokenizer, one_hot, hashing_trick, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.feature_extraction.text import HashingVectorizer
import lightgbm as lgb
from sklearn import pipeline
import gc
from tqdm import tqdm_notebook
import datetime
import dask.dataframe as dd
from sklearn.metrics import recall_score

# Logging

In [2]:
LOGS = 0

In [3]:
if LOGS:
    import logging

    if not os.path.exists('../logs/'):
        os.makedirs('../logs/')

    NAME = 'Exploracion'

    LOG_NAME = '../logs/{}_{}.log'.format(datetime.datetime.now().strftime("%Y%m%d"), NAME)
    logging.basicConfig(filename=LOG_NAME, level=logging.WARNING, format='%(asctime)s %(message)s')

    logging.warning("")
    logging.warning("Comienzo script")

# Datos

In [4]:
# train = pd.read_csv('../data/train.csv', usecols=['comment_text', 'target'], nrows=250000)

In [5]:
# train = dd.read_csv('../data/train.csv', usecols=['comment_text', 'target']).head(50000)
train = dd.read_csv('../data/train.csv').head(250000)

In [6]:
train.shape

(139857, 45)

In [7]:
train.head(10)

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,0.000000,"This is so cool. It's like, 'would you want yo...",0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.000000,0,4
1,59849,0.000000,Thank you!! This would make my life a lot less...,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.000000,0,4
2,59852,0.000000,This is such an urgent design problem; kudos t...,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.000000,0,4
3,59855,0.000000,Is this something I'll be able to install on m...,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.000000,0,4
4,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.000000,0.021277,0.872340,0.0,0.0,0.0,...,2006,rejected,0,0,0,1,0,0.000000,4,47
5,59859,0.666667,ur a sh*tty comment.,0.047619,0.638095,0.000000,0.333333,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.009524,0,105
6,59861,0.457627,hahahahahahahahhha suck it.,0.050847,0.305085,0.000000,0.254237,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.220339,0,59
7,59863,0.000000,FFFFUUUUUUUUUUUUUUU,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.000000,0,4
8,239575,0.000000,The ranchers seem motivated by mostly by greed...,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,...,26662,approved,0,0,0,0,0,0.000000,0,4
9,239576,0.000000,It was a great show. Not a combo I'd of expect...,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,...,26650,approved,0,0,0,1,0,0.000000,0,4


In [8]:
identity_columns = [
    'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish',
    'muslim', 'black', 'white', 'psychiatric_or_mental_illness']

In [9]:
text_data = train['comment_text']
text_data.head(10)

0    This is so cool. It's like, 'would you want yo...
1    Thank you!! This would make my life a lot less...
2    This is such an urgent design problem; kudos t...
3    Is this something I'll be able to install on m...
4                 haha you guys are a bunch of losers.
5                                 ur a sh*tty comment.
6                          hahahahahahahahhha suck it.
7                                  FFFFUUUUUUUUUUUUUUU
8    The ranchers seem motivated by mostly by greed...
9    It was a great show. Not a combo I'd of expect...
Name: comment_text, dtype: object

In [10]:
# Overall
weights = np.ones((len(text_data),)) / 4
# Subgroup
weights += (train[identity_columns].fillna(0).values>=0.5).sum(axis=1).astype(bool).astype(np.int) / 4
# Background Positive, Subgroup Negative
weights += (( (train['target'].values>=0.5).astype(bool).astype(np.int) +
   (train[identity_columns].fillna(0).values<0.5).sum(axis=1).astype(bool).astype(np.int) ) > 1 ).astype(bool).astype(np.int) / 4
# Background Negative, Subgroup Positive
weights += (( (train['target'].values<0.5).astype(bool).astype(np.int) +
   (train[identity_columns].fillna(0).values>=0.5).sum(axis=1).astype(bool).astype(np.int) ) > 1 ).astype(bool).astype(np.int) / 4
loss_weight = 1.0 / weights.mean()
# weights = np.vstack([(train['target'].values>=0.5).astype(np.int),weights]).T

In [12]:
weights[:5]

array([0.25, 0.25, 0.25, 0.25, 0.5 ])

Conteo

In [13]:
arrBad = [
'acrotomophilia',
'anal',
'anilingus',
'anus',
'arsehole',
'ass',
'asshole',
'assmunch',
'autoerotic',
'babeland',
'bangbros',
'bareback',
'barenaked',
'bastardo',
'bastinado',
'bbw',
'bdsm',
'bestiality',
'bimbos',
'birdlock',
'bitch',
'blumpkin',
'bollocks',
'bondage',
'boner',
'boob',
'boobs',
'bukkake',
'bulldyke',
'bullet vibe',
'bung hole',
'bunghole',
'busty',
'butt',
'buttcheeks',
'butthole',
'camgirl',
'camslut',
'camwhore',
'carpetmuncher',
'circlejerk',
'clit',
'clitoris',
'clusterfuck',
'cock',
'cocks',
'coprolagnia',
'coprophilia',
'cornhole',
'cum',
'cumming',
'cunnilingus',
'cunt',
'darkie',
'daterape',
'deepthroat',
'dick',
'dildo',
'doggiestyle',
'doggystyle',
'dolcett',
'domination',
'dominatrix',
'dommes',
'ecchi',
'ejaculation',
'erotic',
'erotism',
'escort',
'eunuch',
'faggot',
'fecal',
'felch',
'fellatio',
'feltch',
'femdom',
'figging',
'fingering',
'fisting',
'footjob',
'frotting',
'fuck',
'fucking',
'fudgepacker',
'futanari',
'gay',
'genitals',
'goatcx',
'goatse',
'gokkun',
'goodpoop',
'goregasm',
'grope',
'guro',
'handjob',
'hardcore',
'hentai',
'homoerotic',
'honkey',
'hooker',
'kill',
'murder',
'fat',
'humping',
'incest',
'intercourse',
'jack',
'jerk',
'jigaboo',
'jiggaboo',
'jiggerboo',
'jizz',
'juggs',
'kike',
'kinbaku',
'kinkster',
'kinky',
'knobbing',
'lolita',
'lovemaking',
'masturbate',
'motherfucker',
'muffdiving',
'nambla',
'nawashi',
'negro',
'neonazi',
'nigga',
'nigger',
'nimphomania',
'nipple',
'nipples',
'nude',
'nudity',
'nympho',
'nymphomania',
'octopussy',
'omorashi',
'orgasm',
'orgy',
'paedophile',
'panties',
'panty',
'pedobear',
'pedophile',
'pegging',
'penis',
'pissing',
'pisspig',
'playboy',
'ponyplay',
'poof',
'poopchute',
'porn',
'porno',
'pornography',
'pthc',
'pubes',
'pussy',
'queaf',
'raghead',
'rape',
'raping',
'rapist',
'rectum',
'cowgirl',
'rimjob',
'rimming',
'sadism',
'scat',
'schlong',
'scissoring',
'semen',
'sex',
'sexo',
'sexy',
'beaver',
'pussy',
'shemale',
'shibari',
'shit',
'shota',
'shrimping',
'slanteye',
'slut',
'smut',
'snatch',
'snowballing',
'sodomize',
'sodomy',
'spic',
'spooge',
'strapon',
'strappado',
'strip',
'suck',
'sucks',
'suicide',
'sultry',
'swastika',
'swinger',
'threesome',
'throating',
'tit',
'tits',
'titties',
'titty',
'topless',
'tosser',
'towelhead',
'tranny',
'tribadism',
'tubgirl',
'tushy',
'twat',
'twink',
'twinkie',
'undressing',
'upskirt',
'urophilia',
'vagina',
'vibrator',
'vorarephilia',
'voyeur',
'vulva',
'wank',
'wetback',
'xx',
'xxx',
'yaoi',
'yiffy',
]

In [14]:
arrGood = ['absolutely',
 'abundant',
 'accept',
 'acclaimed',
 'accomplishment',
 'achievement',
 'action',
 'active',
 'activist',
 'acumen',
 'adjust',
 'admire',
 'adopt',
 'adorable',
 'adored',
 'adventure',
 'affirmation',
 'affirmative',
 'affluent',
 'agree',
 'airy',
 'alive',
 'alliance',
 'ally',
 'alter',
 'amaze',
 'amity',
 'animated',
 'answer',
 'appreciation',
 'approve',
 'aptitude',
 'artistic',
 'assertive',
 'astonish',
 'astounding',
 'astute',
 'attractive',
 'authentic',
 'basic',
 'beaming',
 'beautiful',
 'believe',
 'benefactor',
 'benefit',
 'bighearted',
 'blessed',
 'bliss',
 'bloom',
 'bountiful',
 'bounty',
 'brave',
 'bright',
 'brilliant',
 'bubbly',
 'bunch',
 'burgeon',
 'calm',
 'care',
 'celebrate',
 'certain',
 'change',
 'character',
 'charitable',
 'charming',
 'cheer',
 'cherish',
 'clarity',
 'classy',
 'clean',
 'clever',
 'closeness',
 'commend',
 'companionship',
 'complete',
 'comradeship',
 'confident',
 'connect',
 'connected',
 'constant',
 'content',
 'conviction',
 'copious',
 'core',
 'coupled',
 'courageous',
 'creative',
 'cuddle',
 'cultivate',
 'cure',
 'curious',
 'cute',
 'dazzling',
 'delight',
 'direct',
 'discover',
 'distinguished',
 'divine',
 'donate',
 'each day',
 'eager',
 'earnest',
 'easy',
 'ecstasy',
 'effervescent',
 'efficient',
 'effortless',
 'electrifying',
 'elegance',
 'embrace',
 'encompassing',
 'encourage',
 'endorse',
 'energized',
 'energy',
 'enjoy',
 'enormous',
 'enthuse',
 'enthusiastic',
 'entirely',
 'essence',
 'established',
 'esteem',
 'everyday',
 'everyone',
 'excited',
 'exciting',
 'exhilarating',
 'expand',
 'explore',
 'express',
 'exquisite',
 'exultant',
 'faith',
 'familiar',
 'family',
 'famous',
 'feat',
 'fit',
 'flourish',
 'fortunate',
 'fortune',
 'freedom',
 'fresh',
 'friendship',
 'full',
 'funny',
 'gather',
 'generous',
 'genius',
 'genuine',
 'give',
 'glad',
 'glow',
 'good',
 'gorgeous',
 'grace',
 'graceful',
 'gratitude',
 'green',
 'grin',
 'group',
 'grow',
 'handsome',
 'happy',
 'harmony',
 'healed',
 'healing',
 'healthful',
 'healthy',
 'heart',
 'hearty',
 'heavenly',
 'helpful',
 'here',
 'hold',
 'holy',
 'honest',
 'honor',
 'hug',
 'idea',
 'ideal',
 'imaginative',
 'increase',
 'incredible',
 'independent',
 'ingenious',
 'innate',
 'innovate',
 'inspire',
 'instantaneous',
 'instinct',
 'intellectual',
 'intelligence',
 'intuitive',
 'inventive',
 'joined',
 'jovial',
 'joy',
 'jubilation',
 'keen',
 'key',
 'kind',
 'kiss',
 'knowledge',
 'laugh',
 'leader',
 'learn',
 'legendary',
 'light',
 'lively',
 'love',
 'loveliness',
 'lucidity',
 'lucrative',
 'luminous',
 'maintain',
 'marvelous',
 'master',
 'meaningful',
 'meditate',
 'mend',
 'metamorphosis',
 'mind-blowing',
 'miracle',
 'mission',
 'modify',
 'motivate',
 'moving',
 'natural',
 'nature',
 'nourish',
 'nourished',
 'novel',
 'now',
 'nurture',
 'nutritious',
 'one',
 'open',
 'openhanded',
 'optimistic',
 'paradise',
 'party',
 'peace',
 'perfect',
 'phenomenon',
 'pleasure',
 'plenteous',
 'plentiful',
 'plenty',
 'plethora',
 'poise',
 'polish',
 'popular',
 'positive',
 'powerful',
 'prepared',
 'pretty',
 'principle',
 'productive',
 'project',
 'prominent',
 'prosperous',
 'protect',
 'proud',
 'purpose',
 'quest',
 'quick',
 'quiet',
 'ready',
 'recognize',
 'refinement',
 'refresh',
 'rejoice',
 'rejuvenate',
 'relax',
 'reliance',
 'rely',
 'remarkable',
 'renew',
 'renowned',
 'replenish',
 'resolution',
 'resound',
 'resources',
 'respect',
 'restore',
 'revere',
 'revolutionize',
 'rewarding',
 'rich',
 'robust',
 'rousing',
 'safe',
 'secure',
 'see',
 'sensation',
 'serenity',
 'shift',
 'shine',
 'show',
 'silence',
 'simple',
 'sincerity',
 'smart',
 'smile',
 'smooth',
 'solution',
 'soul',
 'sparkling',
 'spirit',
 'spirited',
 'spiritual',
 'splendid',
 'spontaneous',
 'still',
 'stir',
 'strong',
 'style',
 'success',
 'sunny',
 'support',
 'sure',
 'surprise',
 'sustain',
 'synchronized',
 'team',
 'thankful',
 'therapeutic',
 'thorough',
 'thrilled',
 'thrive',
 'today',
 'together',
 'tranquil',
 'transform',
 'triumph',
 'trust',
 'truth',
 'unity',
 'unusual',
 'unwavering',
 'upbeat',
 'value',
 'vary',
 'venerate',
 'venture',
 'very',
 'vibrant',
 'victory',
 'vigorous',
 'vision',
 'visualize',
 'vital',
 'vivacious',
 'voyage',
 'wealthy',
 'welcome',
 'well',
 'whole',
 'wholesome',
 'willing',
 'wonder',
 'wonderful',
 'wondrous',
 'xanadu',
 'yes',
 'yippee',
 'young',
 'youth',
 'youthful',
 'zeal',
 'zest',
 'zing',
 'zip']

In [15]:
%%time
count_ast = [phrase.count('*') for phrase in text_data]
count_ex = [phrase.count('!') for phrase in text_data]
count_qu = [phrase.count('?') for phrase in text_data]
count_ar = [phrase.count('@') for phrase in text_data]
count_ha = [phrase.count('#') for phrase in text_data]
len_pr = [len(phrase) for phrase in text_data]
len_max_word = [max([len(x) for x in phrase.split()]) for phrase in text_data]
len_min_word = [min([len(x) for x in phrase.split()]) for phrase in text_data]

CPU times: user 1.58 s, sys: 2.64 ms, total: 1.58 s
Wall time: 1.58 s


In [16]:
data_extra = pd.DataFrame({'ast': count_ast,
                          'ex': count_ex,
                          'qu': count_qu,
                           'ar': count_ar,
                          'ha': count_ha,
                          'len_pr': len_pr,
                          'len_max_word': len_max_word,
                          'len_min_word': len_min_word})

data_extra.head(10)

,ast,ex,qu,ar,ha,len_pr,len_max_word,len_min_word
0,0,1,2,0,0,101,7,2
1,0,3,0,0,0,114,17,1
2,0,1,0,0,0,86,11,2
3,0,0,2,0,0,84,9,2
4,0,0,0,0,0,36,7,1
5,1,0,0,0,0,20,8,1
6,0,0,0,0,0,27,18,3
7,0,0,0,0,0,19,19,19
8,0,0,0,0,0,120,9,2
9,0,0,0,0,0,80,8,1


Data Cleaning

In [17]:
CHARS_TO_REMOVE = '!¡"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n“”’\'∞θ÷α•à−β∅³π‘₹´°£€\×™√²—'
MAX_LEN = 2000
stop_words = list(stopwords.words('english'))

In [18]:
def preprocess(data, sw):
    def clean_special_chars(text, sw):
        text = ' '.join([word for word in text.split() if word.lower() not in sw])
        return text

    data = data.astype(str).apply(lambda x: clean_special_chars(x, sw))
    return data

In [19]:
def preprocess2(data, chars):
    def clean_special_chars2(text, chars):
        text = ''.join([word for word in text if word not in chars])
        return text

    data = data.astype(str).apply(lambda x: clean_special_chars2(x, chars))
    return data

In [20]:
%%time
text_data_clean = preprocess(text_data, stop_words)

CPU times: user 8.51 s, sys: 26 ms, total: 8.54 s
Wall time: 8.57 s


In [21]:
%%time
text_data_clean2 = preprocess2(text_data_clean, CHARS_TO_REMOVE)

CPU times: user 1.38 s, sys: 7.89 ms, total: 1.39 s
Wall time: 1.39 s


In [22]:
# text_data_clean2[:5]

# list_len = [len(x.split()) for x in text_data_clean2]
# maximo = max(list_len)
# print(maximo)

# def preprocess3(data, m):
#     def clean_special_chars3(text, m):
#         text = one_hot(text, m)
#         return text

#     data = data.astype(str).apply(lambda x: clean_special_chars3(x, m))
#     return data

# %%time
# text_data_clean3 = preprocess3(text_data_clean2, maximo)

# text_data_clean3[:5]

In [23]:
%%time
bad_words = [len(' '.join([word for word in i.split() if word.lower() in arrBad])) for i in text_data_clean2]
data_extra['bad_words'] = bad_words

CPU times: user 8.14 s, sys: 3.37 ms, total: 8.14 s
Wall time: 8.14 s


In [24]:
%%time
good_words = [len(' '.join([word for word in i.split() if word.lower() in arrGood])) for i in text_data_clean2]
data_extra['good_words'] = good_words

CPU times: user 12.8 s, sys: 7.53 ms, total: 12.8 s
Wall time: 12.8 s


In [25]:
data_extra['bad_ratio'] = data_extra.bad_words / data_extra.len_pr
data_extra['good_ratio'] = data_extra.good_words / data_extra.len_pr
data_extra['bad_plus_good'] = data_extra.bad_words + data_extra.good_words

In [26]:
data_extra.head(10)

,ast,ex,qu,ar,ha,len_pr,len_max_word,len_min_word,bad_words,good_words,bad_ratio,good_ratio,bad_plus_good
0,0,1,2,0,0,101,7,2,0,9,0.000000,0.089109,9
1,0,3,0,0,0,114,17,1,0,0,0.000000,0.000000,0
2,0,1,0,0,0,86,11,2,0,0,0.000000,0.000000,0
3,0,0,2,0,0,84,9,2,0,0,0.000000,0.000000,0
4,0,0,0,0,0,36,7,1,0,5,0.000000,0.138889,5
5,1,0,0,0,0,20,8,1,0,0,0.000000,0.000000,0
6,0,0,0,0,0,27,18,3,4,0,0.148148,0.000000,4
7,0,0,0,0,0,19,19,19,0,0,0.000000,0.000000,0
8,0,0,0,0,0,120,9,2,0,3,0.000000,0.025000,3
9,0,0,0,0,0,80,8,1,0,18,0.000000,0.225000,18


In [27]:
vectorizer = HashingVectorizer(n_features=MAX_LEN)

In [28]:
# vectorizer = TfidfVectorizer(max_features=MAX_LEN).fit(text_data_clean2)

In [29]:
text_data_clean3 = vectorizer.transform(text_data_clean2).toarray()

In [30]:
X_words = pd.DataFrame(text_data_clean3)

In [31]:
# a = [len(X_words.iloc[:,i].value_counts()) for i in range(X_words.shape[1])]
# sel_cols = list()
# thr = 0.25

# MAX = max(a)

# for i in range(X_words.shape[1]):
#     vc = len(X_words.iloc[:,i].value_counts())
#     if vc/MAX < thr:
#         sel_cols.append(i)
        
# print(len(sel_cols))

# X_words2 = X_words.iloc[:,sel_cols]

In [32]:
X_train = pd.concat([X_words, data_extra], axis=1)

del X_words
del data_extra
gc.collect()

42

In [33]:
X_train.head()

,0,1,2,3,4,5,6,7,8,9,...,ar,ha,len_pr,len_max_word,len_min_word,bad_words,good_words,bad_ratio,good_ratio,bad_plus_good
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,101,7,2,0,9,0.0,0.089109,9
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,114,17,1,0,0,0.0,0.000000,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,86,11,2,0,0,0.0,0.000000,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,84,9,2,0,0,0.0,0.000000,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,36,7,1,0,5,0.0,0.138889,5


In [34]:
# tokenizer = Tokenizer(lower=True)

# %%time
# tokenizer.fit_on_texts(text_data_clean2)

# %%time
# X_train = tokenizer.texts_to_sequences(text_data_clean2)

# %%time
# X_train = pad_sequences(X_train, maxlen=maximo)

# X_train = pd.DataFrame(X_train)

# X_train.head(10)

# drop_cols = [i for i in range(MAX_LEN) if len(X_train.iloc[:,i].value_counts()) < 5000]

# drop_cols = list()
# for i in range(MAX_LEN):
#     lgth = len(X_train.iloc[:,i].value_counts())
#     if lgth == 1:
#         drop_cols.append(str(i))

# X_train.drop(drop_cols, axis=1, inplace=True)

In [35]:
# X_train = X_train[ft_sel]
# X_train.shape

In [79]:
y_train = np.where(train['target'] >= 0.5, 1, 0)

FT sel

In [83]:
k = 3

train_ids = X_train.index
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
skf.get_n_splits(train_ids, y_train)

3

In [84]:
params={
    'min_data_in_leaf':20,
        'max_depth':-1,
        'metric':'auc',
        'n_estimators':1000,
        'learning_rate':0.07,
        'num_leaves':75,
        'colsample_bytree':0.7,
        'objective':'binary',
        'n_jobs':-1,
        'seed':42,
        'bagging_fraction':0.5,
        'lambda_l1':0,
        'lambda_l2':0,
    'metric_freq':50}

In [85]:
lgb_model = lgb.LGBMClassifier(**params)

ft_importances = np.zeros(X_train.shape[1])

counter = 1
for train_index, test_index in skf.split(train_ids, y_train):
    print('Fold {}\n'.format(counter))

    X_fit, X_val = X_train.iloc[train_index, :], X_train.iloc[test_index, :]
    y_fit, y_val = y_train[train_index], y_train[test_index]

    lgb_model.fit(X_fit,
                  y_fit,
                  eval_set=[(X_val, y_val)],
                  verbose=50,
                  early_stopping_rounds=50)

    del X_fit
    del X_val
    del y_fit
    del y_val
    del train_index
    del test_index
    gc.collect()

    ft_importances += lgb_model.feature_importances_

    counter += 1

Fold 1

Training until validation scores don't improve for 50 rounds.
[50]	valid_0's auc: 0.860632
[100]	valid_0's auc: 0.869627
[150]	valid_0's auc: 0.870354
Early stopping, best iteration is:
[126]	valid_0's auc: 0.870613
Fold 2

Training until validation scores don't improve for 50 rounds.
[50]	valid_0's auc: 0.862622
[100]	valid_0's auc: 0.869741
[150]	valid_0's auc: 0.870172
Early stopping, best iteration is:
[125]	valid_0's auc: 0.870694
Fold 3

Training until validation scores don't improve for 50 rounds.
[50]	valid_0's auc: 0.864851
[100]	valid_0's auc: 0.871041
[150]	valid_0's auc: 0.870198
Early stopping, best iteration is:
[108]	valid_0's auc: 0.871245


In [86]:
columnas = X_train_red.columns
imp = pd.DataFrame({'feature': columnas, 'importance': ft_importances/k})
df_imp_sort = imp.sort_values('importance', ascending=False)

df_imp_sort.head(30)

,feature,importance
2005,len_pr,146.666667
1521,1521,96.000000
2006,len_max_word,81.666667
2011,good_ratio,76.333333
608,608,75.000000
910,910,61.666667
1165,1165,55.666667
1312,1312,55.666667
950,950,55.333333
1214,1214,54.666667


Model

In [87]:
ft_sel = df_imp_sort['feature'].values[:500]

In [88]:
k = 3
X_sel = X_train.loc[:,ft_sel]
train_ids = X_sel.index
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
skf.get_n_splits(train_ids, y_train)

3

In [89]:
params={
    'min_data_in_leaf':20,
        'max_depth':-1,
        'metric':'auc',
        'n_estimators':1000,
        'learning_rate':0.07,
        'num_leaves':75,
        'colsample_bytree':0.3,
        'objective':'binary',
        'n_jobs':-1,
        'seed':42,
        'bagging_fraction':0.8,
        'lambda_l1':0,
        'lambda_l2':0,
    'metric_freq':50}

In [90]:
def recall(preds, train_data):
    r = recall_score([1 if i >=0.5 else 0 for i in train_data], preds, average='macro')  
    return 'recall', r, True

In [91]:
lgb_model = lgb.LGBMClassifier(**params)

ft_importances = np.zeros(X_sel.shape[1])

counter = 1
for train_index, test_index in skf.split(train_ids, y_train):
    print('Fold {}\n'.format(counter))

    X_fit, X_val = X_sel.iloc[train_index, :], X_sel.iloc[test_index, :]
    y_fit, y_val = y_train[train_index], y_train[test_index]
    
    we = weights[train_index]

    lgb_model.fit(X_fit,
                  y_fit,
                  eval_set=[(X_val, y_val)],
                  verbose=50,
                  early_stopping_rounds=50,
#                  eval_metric=recall
                 )

    del X_fit
    del X_val
    del y_fit
    del y_val
    del train_index
    del test_index
    gc.collect()

    ft_importances += lgb_model.feature_importances_

    counter += 1

Fold 1

Training until validation scores don't improve for 50 rounds.
[50]	valid_0's auc: 0.86476
[100]	valid_0's auc: 0.873429
[150]	valid_0's auc: 0.874694
[200]	valid_0's auc: 0.873697
Early stopping, best iteration is:
[161]	valid_0's auc: 0.874863
Fold 2

Training until validation scores don't improve for 50 rounds.
[50]	valid_0's auc: 0.866776
[100]	valid_0's auc: 0.873929
[150]	valid_0's auc: 0.875319
[200]	valid_0's auc: 0.874588
Early stopping, best iteration is:
[154]	valid_0's auc: 0.875442
Fold 3

Training until validation scores don't improve for 50 rounds.
[50]	valid_0's auc: 0.867859
[100]	valid_0's auc: 0.875879
[150]	valid_0's auc: 0.876154
Early stopping, best iteration is:
[111]	valid_0's auc: 0.876639


In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import EarlyStopping
from keras.optimizers import SGD
import tensorflow as tf
from sklearn.metrics import roc_auc_score

In [ ]:
def auroc(y_true, y_pred):
    return tf.py_func(roc_auc_score, (y_true, y_pred), tf.double)

In [ ]:
callbacks = [EarlyStopping(monitor='val_auroc',
                           min_delta=0.0,
                           patience=1,
                           verbose=0,
                           mode='max',
                           restore_best_weights=True)]

In [ ]:
def create_baseline():
    model = Sequential()
    model.add(Dense(64, input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))
    model.add(Dense(256, kernel_initializer='normal', activation='relu'))
#     model.add(Dense(1024, kernel_initializer='normal', activation='relu'))
#     model.add(Dense(512, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam',metrics=[auroc])
    return model

In [ ]:
estimator = KerasClassifier(build_fn=create_baseline, epochs=30, batch_size=1024, verbose=1, validation_split=0.33,
                           callbacks=callbacks)

In [ ]:
model = create_baseline()

In [ ]:
estimator.fit(X_train, y_train)